In [7]:
import sys
sys.path.append("..")
import tools
import tensorflow as tf
import numpy as np
import json
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
repo = "/epyc/ssd/users/kmrakovc/DATA/rc2_subset/SMALL_HSC/"
output_coll = "u/kmrakovc/single_frame_injection_01"
tools.data.convert_butler_tfrecords (repo, output_coll, shape=(128,128), 
                                     filename_train="/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/train.tfrecord",
                                     filename_test="/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test.tfrecord")

inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)(4176, 2048)
 (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)inside split: 
(4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048)  (528, 128, 128)(528, 128, 128)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 2048)(4176, 2048) 
(4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 20

inside split: (4224, 2048)(4176, 2048)
 (4176, 2048) (528, 128, 128) (528, 128, 128)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split:inside split:  (4224, 2048)(4224, 2048)inside split:

 (4224, 2048)
inside split: (4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
(4176, 2048)inside split:  (4224, 2048)(4176, 2048) 
(528, 128, 128) (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)inside split:inside split:
 (4224, 2048) 
(4224, 2048)
(4176, 2048) (4176, 2048) (528, 128, 128) (528, 128, 128)
inside split:inside split:  (4224, 2048)(4224, 2048)

inside split: (4224, 2048)(4176, 2048) 
(4176, 2048)(4176, 2048)  (528, 128, 128)(4176, 2048)  (528, 128, 128)(528, 128, 128)
 (528, 128, 128)
inside split: (4224, 2048)
inside split: (4224, 2048)
inside split: (4224, 20

NameError: name 'brea' is not defined

Process ForkPoolWorker-82:
Process ForkPoolWorker-85:
Process ForkPoolWorker-46:
Process ForkPoolWorker-74:
Process ForkPoolWorker-62:
Process ForkPoolWorker-67:
Process ForkPoolWorker-81:
Process ForkPoolWorker-64:
Process ForkPoolWorker-59:
Process ForkPoolWorker-66:
Process ForkPoolWorker-61:
Process ForkPoolWorker-63:
Process ForkPoolWorker-57:
Process ForkPoolWorker-50:
Process ForkPoolWorker-56:
Process ForkPoolWorker-89:
Process ForkPoolWorker-1:
Process ForkPoolWorker-23:
Process ForkPoolWorker-16:
Process ForkPoolWorker-20:
Process ForkPoolWorker-25:
Process ForkPoolWorker-71:
Process ForkPoolWorker-21:
Process ForkPoolWorker-86:
Process ForkPoolWorker-84:
Process ForkPoolWorker-95:
Process ForkPoolWorker-55:
Process ForkPoolWorker-53:
Process ForkPoolWorker-76:
Process ForkPoolWorker-29:
Process ForkPoolWorker-12:
Process ForkPoolWorker-94:
Process ForkPoolWorker-54:
Process ForkPoolWorker-75:
Process ForkPoolWorker-91:
Process ForkPoolWorker-3:
Process ForkPoolWorker-42:
Pro

Traceback (most recent call last):
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 314, i

...

  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/

  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  Fi

  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._r

  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/astro/

  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
KeyboardInterrupt
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
KeyboardInterrupt
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/astro/users/kmrakovc/miniconda3/envs/lsst-scipipe-8.0.0/lib/python3.11/multiprocess

In [3]:
dataset_train = tf.data.TFRecordDataset(["/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/train.tfrecord"])
tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_train)
dataset_train = dataset_train.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
#dataset_train = dataset_train.batch(128).prefetch(2)
dataset_val = tf.data.TFRecordDataset(["/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/test.tfrecord"])
dataset_val = dataset_val.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
#dataset_val = dataset_val.batch(128).prefetch(2)

2024-03-26 13:27:42.805065: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [380]:
print ("Train dataset:")
for i,j in enumerate(dataset_train):
    print ("\r",i, end="")
print ("\nTest dataset:")
for i,j in enumerate(dataset_val):
    print ("\r",i, end="")

Train dataset:
 92927
Test dataset:
 30623

In [12]:
dataset_train_merged = tools.data.merge (dataset_train, (4176, 2048))
dataset_val_merged = tools.data.merge (dataset_val, (4176, 2048))

In [13]:
def dataset_to_numpy(dataset):
    for i, a in enumerate(dataset):
        s = a.shape
    array = np.empty((i+1)+s)
    for i, a in enumerate(dataset):
        array[i] = a
    return array
test = dataset_to_numpy(dataset_train_merged)
val = dataset_to_numpy(dataset_val_merged)

In [15]:
test.shape, val.shape

((176, 4176, 2048), (58, 4176, 2048))

In [17]:
tools.data.get_asteroid_num(test[1])

20